In [1]:
import datetime
import time
import calendar
from backend.utility import *
user = User.objects.get(email="veyorokon@gmail.com")
invoice = Invoice.objects.filter(user=user)[0]
invoice.save()
print(invoice.user)

Email=veyorokon@gmail.com


In [2]:
member = SubscriptionMember.objects.filter(user=user)[0]
stripeInvoice = invoice.get_stripe_invoice()
print(stripeInvoice)

{
  "amount_due": 6958,
  "amount_paid": 0,
  "amount_remaining": 6958,
  "application_fee": null,
  "attempt_count": 0,
  "attempted": false,
  "billing": "charge_automatically",
  "billing_reason": "upcoming",
  "charge": null,
  "created": 1554102000,
  "currency": "usd",
  "custom_fields": null,
  "customer": "cus_Ee5GDE9rGJ7uW6",
  "date": 1554102000,
  "default_source": null,
  "description": null,
  "discount": null,
  "due_date": null,
  "ending_balance": 0,
  "finalized_at": null,
  "footer": null,
  "lines": {
    "data": [
      {
        "amount": 250,
        "currency": "usd",
        "description": "Anorak Management Fee",
        "discountable": true,
        "id": "ii_1EElmuGjkoMLHlzN1ksuH77U",
        "invoice_item": "ii_1EElmuGjkoMLHlzN1ksuH77U",
        "livemode": false,
        "metadata": {},
        "object": "line_item",
        "period": {
          "end": 1554076800,
          "start": 1552778371
        },
        "plan": null,
        "proration": false,
  

In [8]:
def get_item_from_invoice(plan, invoice):
    stripeInvoice = invoice.get_stripe_invoice()
    item = invoice.find_items(plan.split(), stripeInvoice)[0]
    return item, stripeInvoice

def is_prorated(stripeInvoiceItem):
    itemPrice = stripeInvoiceItem.amount
    planPrice = stripeInvoiceItem.plan.amount
    return itemPrice != planPrice

def days_in_a_month(date):
    return calendar.monthrange(date.year,date.month)[1]

def get_receipt_data(member, invoice):
    total = 0
    output = {}
    
    plan = member.subscription_account.price_plan.get_product_name()
    item, stripeInvoice = get_item_from_invoice(plan, invoice)
    itemStartEpoch = item.period.start
    if is_prorated(item):
        amount = item.amount/100
        output["items"]=[{"description": item.description,  "price": amount}]
        total += amount
    amount = item.plan.amount/100
    
    output["total"] = (total + amount)
    
    date = convert_epoch(stripeInvoice.period_end)
    monthOf = convert_epoch(stripeInvoice.period_end)
    lastDay = days_in_a_month(date)
    endDateTime = datetime.strptime('{0} {1} {2}'.format(date.month, lastDay, date.year), '%m %d %Y')
    output["date_end"] = endDateTime.strftime('%B %d, %Y')
    output["date_renew"] = get_first_day_of_next_month(endDateTime).strftime('%B %d, %Y')
    output["date_start"] = convert_epoch(itemStartEpoch).strftime('%B %d, %Y')
    output["date_billing"] = monthOf.strftime('%B %d, %Y')
    output["invoice_number"] = stripeInvoice.number
    
    output["lastItem"]={"description": plan+" - Month of "+monthOf.strftime('%B'),  "price": amount}
    output["service"]= member.subscription_account.service.name
    return output

def convert_epoch(epoch):
    date = datetime.fromtimestamp(epoch)
    return date

In [9]:
data = get_receipt_data(member, invoice)
print(data)

{'items': [{'description': 'Remaining time on Netflix - Family Plan (4 users) after 14 Mar 2019', 'price': 8.18}], 'total': 22.17, 'date_end': 'April 30, 2019', 'date_renew': 'May 01, 2019', 'date_start': 'March 14, 2019', 'date_billing': 'April 01, 2019', 'invoice_number': '85FC4AA-0002', 'lastItem': {'description': 'Netflix - Family Plan (4 users) - Month of April', 'price': 13.99}, 'service': 'Netflix'}


In [10]:
from mail_templated import EmailMessage

In [11]:
message = EmailMessage('invoice.tpl', 
                       {'user': user, 'data':data}, 
                       'Anorak@ianorak.com', 
                       to =[user.email]
                      )
message.send()

1

In [4]:
from accounting.email import *

In [5]:
user = User.objects.get(email="veyorokon@gmail.com")
invoice = Invoice.objects.filter(user=user)[0]
member = SubscriptionMember.objects.filter(user=user)[0]

In [6]:
email_receipt(member, invoice)